In [30]:
# importing libraries
import pandas as pd
import requests
from yahooquery import Ticker
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import percentileofscore

In [23]:
# Step 1: Collect the list of Russell 1000 companies and their tickers
def get_russell_1000_tickers():
    url = "https://en.wikipedia.org/wiki/Russell_1000_Index"
    html = requests.get(url).text
    russell_1000_data = pd.read_html(html)[2]  # Adjust the index depending on the table structure
    russell_1000_data['Ticker'] = russell_1000_data['Ticker'].str.replace(".", "-")
    tickers = russell_1000_data['Ticker'].tolist()
    return tickers


# Step 2: Retrieve financial data for each company
def get_financial_data(tickers):
    data = {}
    for ticker in tickers[:12]:
        stock = Ticker(ticker)
        # gather statements
        income_statement = stock.income_statement(
            frequency='a', trailing=True
            )
        
        balance_sheet = stock.balance_sheet(
            frequency='q', trailing=False
            )
        
        cash_flow = stock.cash_flow(
            frequency='a', 
            trailing=True
            )
        
        summary = stock.summary_detail
           
        try:
            market_cap = stock.price[ticker]["marketCap"]
            revenue = income_statement['TotalRevenue'].dropna().iloc[-1]
            fcf = cash_flow['FreeCashFlow'].dropna().iloc[-1]
            net_income = income_statement['NetIncome'].dropna().iloc[-1]
            equity = balance_sheet['StockholdersEquity'].dropna().iloc[-1]
            liabilities = balance_sheet['TotalLiabilitiesNetMinorityInterest'].dropna().iloc[-1]
            total_debt = balance_sheet['TotalDebt'].dropna().iloc[-1]
            

            # Step 3: Calculate the valuation metrics
            try:
                cash_and_equivalents = balance_sheet['CashCashEquivalentsAndShortTermInvestments'].dropna().iloc[-1]
                
                ev = market_cap + total_debt - cash_and_equivalents
                
                op_income = income_statement['OperatingIncome'].dropna().iloc[-1]
                
            except:
                cash_and_equivalents = balance_sheet['CashAndCashEquivalents'].dropna().iloc[-1]
                
                ev = market_cap + total_debt - cash_and_equivalents
                
                op_income = income_statement['PretaxIncome'].dropna().iloc[-1]

            acquirers_multiple = ev / op_income

            ev_to_fcf = ev / fcf

            ev_to_sales = ev / revenue

            graham_test = ((market_cap / net_income) < 10 ) & ((equity > liabilities)) 
            
            try:
                dividend_raw = summary[ticker]["trailingAnnualDividendYield"]
                dividend_yield = dividend_raw * 100
            
            except:
                dividend_yield = 0
                
            try:
                buyback = (cash_flow["RepurchaseOfCapitalStock"].dropna().iloc[-1] + cash_flow["RepurchaseOfCapitalStock"].dropna().iloc[-2]) * -1 / 2
                buyback_yield = buyback * 100 / market_cap
            
            except:
                buyback_yield = 0
            
            data[ticker] = {
                "acquirer's multiple": acquirers_multiple,
                "EV/FCF" : ev_to_fcf, 
                "EV/Sales": ev_to_sales, 
                "P/E" : market_cap / net_income, 
                "Debt/Equity": total_debt / equity, 
                "Passes Graham test": graham_test,
                "Enterprise Value (M)": ev / 1000000, 
                "Market cap (M)": market_cap / 1000000, 
                "Net income (M)": net_income / 1000000, 
                "Operating income (M)": op_income / 1000000, 
                "Dividend Yield (%)" : dividend_yield , 
                "Shareholder Yield (%)": dividend_yield + buyback_yield, 
                "Buyback Yield (%)" : buyback_yield
                }
        
        except Exception as e:
            print(f"Error processing {ticker}: {e}")
    return data

# Step 3: Add value factor based on percentiles
def add_value_factors(df):
    
    # filtering outliers
    df = df[
        ((df["acquirer\'s multiple"] < 200)
        & (df["acquirer\'s multiple"] > 0)
        & (df["Enterprise Value (M)"] > 0)
        & (df["EV/FCF"] > 0)
        & (df["EV/FCF"] < 200)
        & (df["EV/Sales"] > 0)
        & (df["EV/Sales"] < 200)
        & (df["P/E"] > 0)
        & (df["P/E"] < 200))
    ].copy()
    
    # adding the ranks
    ratios = [
        "acquirer\'s multiple", 
        "EV/FCF", 
        "EV/Sales", 
        "P/E", 
        "Dividend Yield (%)", 
        "Shareholder Yield (%)", 
        "Buyback Yield (%)"
        ]

    df_to_normalize = df[ratios]

    scaler = MinMaxScaler(feature_range=(0,100))
    scaler = scaler.fit(df_to_normalize)
    df_normalized = scaler.transform(df_to_normalize)
    column_names = [name + "_rank" for name in scaler.feature_names_in_]

    # adding column names back
    df_normalized = pd.DataFrame(df_normalized, columns=column_names)

    # lower ratio = higher rank (except dividends and buyback)
    df_normalized["acquirer\'s multiple_rank"] = 100 - df_normalized["acquirer\'s   multiple_rank"]
    df_normalized["EV/FCF_rank"] = 100 - df_normalized["EV/FCF_rank"]
    df_normalized["EV/Sales_rank"] = 100 - df_normalized["EV/Sales_rank"]
    df_normalized["P/E_rank"] = 100 - df_normalized["P/E_rank"]
    
    # value factors

    df["VF2"] = (df["acquirer\'s multiple_rank"]
        + df["EV/FCF_rank"]
        + df["EV/Sales_rank"]
        + df["P/E_rank"]
        + df["Shareholder Yield (%)_rank"]
    ) / 5
    
    df["VF2"] = (df["acquirer\'s multiple_rank"]
        + df["EV/FCF_rank"]
        + df["EV/Sales_rank"]
        + df["P/E_rank"]
        + df["Buyback Yield (%)_rank"]
    ) / 5
    
    return df

In [24]:
tickers = get_russell_1000_tickers()
data = get_financial_data(tickers)


# Step 4: Store the results in a DataFrame
df = pd.DataFrame(data).T

# Step 5: Add value factor to the data
df = add_value_factors(df)

# export
df.sort_values(by="VF3", ascending=False, inplace=True)
df.to_excel("value_factor_export.xlsx")

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [35]:
df.sort_values(by="acquirer\'s multiple")

,acquirer's multiple,EV/FCF,EV/Sales,P/E,Debt/Equity,Passes Graham test,Enterprise Value (M),Market cap (M),Net income (M),Operating income (M),Dividend Yield (%),Shareholder Yield (%),Buyback Yield (%)
TXG,-33.611631,-36.241305,11.170299,-36.170453,0.127173,True,5989.491776,6305.739776,-174.334,-178.197,0.0,0.0,0
AYI,10.141253,12.328955,1.281343,13.247614,0.300318,False,5269.395168,5023.495168,379.2,519.6,0.335592,10.920852,10.58526
ADM,11.737849,30.989347,0.499887,9.284218,0.416954,True,50977.476544,41370.476544,4456.0,4343.0,2.22312,5.728035,3.504915
ACHC,16.347382,218.216204,2.91517,22.87665,0.550089,False,7865.16664,6366.96064,278.317,481.127,0.0,0.0,0
ADT,16.454822,15.299813,2.315385,171.385524,2.968627,False,14963.32384,5308.32384,30.973,909.358,0.0,22.606006,22.606006
ACN,17.522283,18.112811,2.610951,24.193983,0.135023,False,164866.95476,167901.42976,6939.801,9408.988,1.624765,4.075891,2.451125
MMM,17.770472,17.082441,2.072008,10.45385,1.056804,False,69269.300096,57015.300096,5454.0,3898.0,5.862137,7.777412,1.915275
ABBV,18.683322,12.998501,5.554848,34.429221,3.667034,False,315187.64096,261145.64096,7585.0,16870.0,3.922367,4.491781,0.569414
ADP,25.175802,26.939712,5.33755,27.299971,0.898993,False,90512.044256,89019.744256,3260.8,3595.2,2.142089,3.73578,1.593691
ATVI,25.549287,23.881203,6.243778,32.157202,0.179518,False,50843.081664,59748.081664,1858.0,1990.0,0.0,0.0,0


In [48]:
ratios = [
    "acquirer\'s multiple", 
    "EV/FCF", 
    "EV/Sales", 
    "P/E", 
    "Dividend Yield (%)", 
    "Buyback Yield (%)"
    ]

df_to_normalize = df[ratios]

scaler = MinMaxScaler(feature_range=(0,100))
scaler = scaler.fit(df_to_normalize)
df_normalized = scaler.transform(df_to_normalize)
column_names = [name + "_rank" for name in scaler.feature_names_in_]

# adding column names back
df_normalized = pd.DataFrame(df_normalized, columns=column_names)

# lower ratio = higher rank (except dividends and buyback)
df_normalized["acquirer\'s multiple_rank"] = 100 - df_normalized["acquirer\'s multiple_rank"]
df_normalized["EV/FCF_rank"] = 100 - df_normalized["EV/FCF_rank"]
df_normalized["EV/Sales_rank"] = 100 - df_normalized["EV/Sales_rank"]
df_normalized["P/E_rank"] = 100 - df_normalized["P/E_rank"]

In [53]:
df_normalized = pd.DataFrame(df_normalized, columns=column_names)
df_normalized["acquirer\'s multiple_rank"] = 100 - df_normalized["acquirer\'s multiple_rank"]
df_normalized["EV/FCF_rank"] = 100 - df_normalized["EV/FCF_rank"]
df_normalized["EV/Sales_rank"] = 100 - df_normalized["EV/Sales_rank"]
df_normalized["P/E_rank"] = 100 - df_normalized["P/E_rank"]

In [54]:
df_normalized

,acquirer's multiple_rank,EV/FCF_rank,EV/Sales_rank,P/E_rank,Dividend Yield (%)_rank,Buyback Yield (%)_rank
0,100.000000,0.000000,100.000000,0.000000,0.000000,0.000000
1,17.765219,20.955855,14.733462,22.463484,100.000000,8.472418
2,0.000000,24.359994,40.680175,33.478270,30.159580,8.681837
3,16.304243,19.350895,47.373629,34.014763,66.910194,2.518862
4,20.042812,100.000000,22.635331,28.448761,0.000000,0.000000
5,18.162434,21.360783,19.784275,29.083449,27.716266,10.842806
6,5.315570,23.627720,53.830076,32.920110,0.000000,0.000000
7,29.975446,19.087768,7.323579,23.809513,5.724741,46.824988
8,27.420166,26.421171,0.000000,21.899958,37.923381,15.504353
9,4.704327,22.741623,77.730472,33.681142,0.000000,16.746523
